<a href="https://colab.research.google.com/github/Jarvis-Geun/DeepLearning-Wiki/blob/main/NLP/seq2seq_tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

`-` Reference
- [[Wikidocs] Sequence-to-Sequence](https://wikidocs.net/24996)

In [1]:
import os
import shutil
import zipfile

import pandas as pd
import tensorflow as tf
import urllib3
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# 코퍼스 데이터 전처리

In [2]:
http = urllib3.PoolManager()
url = 'http://www.manythings.org/anki/fra-eng.zip'
filename = 'fra-eng.zip'
path = os.getcwd()
zipfilename = os.path.join(path, filename)
with http.request('GET', url, preload_content=False) as r, open(zipfilename, 'wb') as out_file:
  shutil.copyfileobj(r, out_file)

with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
  zip_ref.extractall(path)

In [3]:
help(os.getcwd)

Help on built-in function getcwd in module posix:

getcwd()
    Return a unicode string representing the current working directory.



`-` fra.txt 출력

In [4]:
pd.read_csv('fra.txt', names=['src', 'tar', 'lic'], sep='\t')

,src,tar,lic
0,Go.,Va !,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
1,Go.,Marche.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
2,Go.,Bouge !,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
3,Hi.,Salut !,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
4,Hi.,Salut.,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
...,...,...,...
192336,A carbon footprint is the amount of carbon dio...,Une empreinte carbone est la somme de pollutio...,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
192337,Death is something that we're often discourage...,La mort est une chose qu'on nous décourage sou...,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
192338,Since there are usually multiple websites on a...,Puisqu'il y a de multiples sites web sur chaqu...,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
192339,If someone who doesn't know your background sa...,Si quelqu'un qui ne connaît pas vos antécédent...,CC-BY 2.0 (France) Attribution: tatoeba.org #9...


In [5]:
# source (src), target (tar), license (lic) => tab을 기준으로 분리
lines = pd.read_csv('fra.txt', names=['src', 'tar', 'lic'], sep='\t')
# license 열 제거
del lines['lic']
print("전체 샘플의 개수 :", len(lines))

전체 샘플의 개수 : 192341


In [6]:
lines = lines.loc[:, 'src':'tar']
lines = lines[0:60000]  # 전체 샘플 중 60,000개만 저장
lines.sample(10)  # 10개 샘플 출력

,src,tar
34812,How is your hearing?,Comment est votre audition ?
31939,Tom is Mary's hero.,Tom est le héros de Mary.
27064,Who was that girl?,Qui était cette fille ?
9248,You look busy.,Vous avez l'air occupées.
13609,How is it going?,Comment vas-tu ?
26896,What song is this?,C'est quel son ?
24339,I'm not like that.,Je ne suis pas comme ça.
26881,What is the delay?,Quel en est le délai ?
55057,I can come if you want.,"Je peux venir, si vous voulez."
24077,I was quite lucky.,J'ai été assez chanceuse.


In [7]:
type(lines)

pandas.core.frame.DataFrame

`-` lines.sample
- [pandas.DataFrame.sample](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sample.html) $\to$ Return a random sample of items from an axis of object.
- 지정된 개수만큼 랜덤으로 샘플 출력

`-` \<sos> $\to$ \t  
`-` \<eos> $\to$ \n
- 참고 : sos (start of sequence), eos (end of sequence)

In [8]:
# tar 열에 \t와 \n 추가
lines.tar = lines.tar.apply(lambda x : '\t ' + x + ' \n')
lines.sample(10)

,src,tar
56450,I'm fascinated by cats.,\t Je suis fasciné par les chats. \n
37793,They stayed friends.,\t Ils restèrent amis. \n
45041,Tom sat on the couch.,\t Tom s'est assis sur le canapé. \n
27280,You have no fever.,\t Tu n'as pas de fièvre. \n
48452,I don't need advisers.,\t Je n'ai pas besoin de conseillers. \n
43471,My father grows rice.,\t Mon père cultive du riz. \n
35987,I'll fix your wagon.,\t Je réparerai ton chariot. \n
34056,Do you speak French?,\t Sais-tu parler français ? \n
56560,I'm not going that way.,\t Je ne me rends pas par là. \n
33974,Do you enjoy losing?,\t Prends-tu plaisir à perdre ? \n


In [9]:
# help(lines.tar.apply)

In [10]:
lines.src[1000]

'I am sick.'

In [11]:
for line in lines.src[1000]:
  print(line)

I
 
a
m
 
s
i
c
k
.


In [12]:
# 문자 집합(set) 구축
src_vocab = set()
for line in lines.src: # 1줄씩 읽음
  for char in line: # 1개의 문자씩 읽음
    src_vocab.add(char)

tar_vocab = set()
for line in lines.tar:
  for char in line:
    tar_vocab.add(char)

In [13]:
src_vocab_size = len(src_vocab) + 1
tar_vocab_size = len(tar_vocab) + 1
print("src_vocab_size :", src_vocab_size)
print("tar_vocab_size :", tar_vocab_size)

src_vocab_size : 80
tar_vocab_size : 105


`-` 80개의 영어 문자, 105개의 프랑스어 문자

## 인덱스 부여

In [14]:
src_vocab = sorted(list(src_vocab))
tar_vocab = sorted(list(tar_vocab))
print(src_vocab[45:75])
print(tar_vocab[45:75])

['W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
['T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w']


In [15]:
src_to_index = [(word, i+1) for i, word in enumerate(src_vocab)]
print(src_to_index)

[(' ', 1), ('!', 2), ('"', 3), ('$', 4), ('%', 5), ('&', 6), ("'", 7), (',', 8), ('-', 9), ('.', 10), ('/', 11), ('0', 12), ('1', 13), ('2', 14), ('3', 15), ('4', 16), ('5', 17), ('6', 18), ('7', 19), ('8', 20), ('9', 21), (':', 22), ('?', 23), ('A', 24), ('B', 25), ('C', 26), ('D', 27), ('E', 28), ('F', 29), ('G', 30), ('H', 31), ('I', 32), ('J', 33), ('K', 34), ('L', 35), ('M', 36), ('N', 37), ('O', 38), ('P', 39), ('Q', 40), ('R', 41), ('S', 42), ('T', 43), ('U', 44), ('V', 45), ('W', 46), ('X', 47), ('Y', 48), ('Z', 49), ('a', 50), ('b', 51), ('c', 52), ('d', 53), ('e', 54), ('f', 55), ('g', 56), ('h', 57), ('i', 58), ('j', 59), ('k', 60), ('l', 61), ('m', 62), ('n', 63), ('o', 64), ('p', 65), ('q', 66), ('r', 67), ('s', 68), ('t', 69), ('u', 70), ('v', 71), ('w', 72), ('x', 73), ('y', 74), ('z', 75), ('°', 76), ('é', 77), ('’', 78), ('€', 79)]


In [16]:
src_to_index = dict([(word, i+1) for i, word in enumerate(src_vocab)])
tar_to_index = dict([(word, i+1) for i, word in enumerate(tar_vocab)])
print(src_to_index)
print(tar_to_index)

{' ': 1, '!': 2, '"': 3, '$': 4, '%': 5, '&': 6, "'": 7, ',': 8, '-': 9, '.': 10, '/': 11, '0': 12, '1': 13, '2': 14, '3': 15, '4': 16, '5': 17, '6': 18, '7': 19, '8': 20, '9': 21, ':': 22, '?': 23, 'A': 24, 'B': 25, 'C': 26, 'D': 27, 'E': 28, 'F': 29, 'G': 30, 'H': 31, 'I': 32, 'J': 33, 'K': 34, 'L': 35, 'M': 36, 'N': 37, 'O': 38, 'P': 39, 'Q': 40, 'R': 41, 'S': 42, 'T': 43, 'U': 44, 'V': 45, 'W': 46, 'X': 47, 'Y': 48, 'Z': 49, 'a': 50, 'b': 51, 'c': 52, 'd': 53, 'e': 54, 'f': 55, 'g': 56, 'h': 57, 'i': 58, 'j': 59, 'k': 60, 'l': 61, 'm': 62, 'n': 63, 'o': 64, 'p': 65, 'q': 66, 'r': 67, 's': 68, 't': 69, 'u': 70, 'v': 71, 'w': 72, 'x': 73, 'y': 74, 'z': 75, '°': 76, 'é': 77, '’': 78, '€': 79}
{'\t': 1, '\n': 2, ' ': 3, '!': 4, '"': 5, '$': 6, '%': 7, '&': 8, "'": 9, '(': 10, ')': 11, ',': 12, '-': 13, '.': 14, '0': 15, '1': 16, '2': 17, '3': 18, '4': 19, '5': 20, '6': 21, '7': 22, '8': 23, '9': 24, ':': 25, '?': 26, 'A': 27, 'B': 28, 'C': 29, 'D': 30, 'E': 31, 'F': 32, 'G': 33, 'H': 3

## 정수 인코딩

### 영어 문장 $\to$ 정수 인코딩
- 인코더의 입력

In [17]:
lines.src

0                            Go.
1                            Go.
2                            Go.
3                            Hi.
4                            Hi.
                  ...           
59995    Why would they do that?
59996    Why would you ask that?
59997    Why would you ask that?
59998    Why would you say that?
59999    Why would you say that?
Name: src, Length: 60000, dtype: object

In [18]:
print(lines.src[0])
print(lines.src[0][0])

Go.
G


In [19]:
src_to_index[lines.src[0][0]]

30

`-` 실제로 30번째 인덱스를 출력해보면 `('G', 30)`이 저장된 것을 확인할 수 있다.

In [20]:
list(src_to_index.items())[29]

('G', 30)

In [21]:
encoder_input = []

# 1개의 문장
for line in lines.src:
  encoded_line = []
  # 각 줄에서 1개의 char
  for char in line:
    # 각 char을 정수로 변환
    encoded_line.append(src_to_index[char])
  encoder_input.append(encoded_line)
print("샘플 => encoder_input[:5] :", encoder_input[:5])

샘플 => encoder_input[:5] : [[30, 64, 10], [30, 64, 10], [30, 64, 10], [31, 58, 10], [31, 58, 10]]


In [22]:
# encoder_input

### 프랑스어 문장 $\to$ 정수 인코딩
- 디코더의 입력

In [23]:
print(tar_to_index)

{'\t': 1, '\n': 2, ' ': 3, '!': 4, '"': 5, '$': 6, '%': 7, '&': 8, "'": 9, '(': 10, ')': 11, ',': 12, '-': 13, '.': 14, '0': 15, '1': 16, '2': 17, '3': 18, '4': 19, '5': 20, '6': 21, '7': 22, '8': 23, '9': 24, ':': 25, '?': 26, 'A': 27, 'B': 28, 'C': 29, 'D': 30, 'E': 31, 'F': 32, 'G': 33, 'H': 34, 'I': 35, 'J': 36, 'K': 37, 'L': 38, 'M': 39, 'N': 40, 'O': 41, 'P': 42, 'Q': 43, 'R': 44, 'S': 45, 'T': 46, 'U': 47, 'V': 48, 'W': 49, 'X': 50, 'Y': 51, 'Z': 52, 'a': 53, 'b': 54, 'c': 55, 'd': 56, 'e': 57, 'f': 58, 'g': 59, 'h': 60, 'i': 61, 'j': 62, 'k': 63, 'l': 64, 'm': 65, 'n': 66, 'o': 67, 'p': 68, 'q': 69, 'r': 70, 's': 71, 't': 72, 'u': 73, 'v': 74, 'w': 75, 'x': 76, 'y': 77, 'z': 78, '\xa0': 79, '«': 80, '»': 81, 'À': 82, 'Ç': 83, 'É': 84, 'Ê': 85, 'Ô': 86, 'à': 87, 'â': 88, 'ç': 89, 'è': 90, 'é': 91, 'ê': 92, 'ë': 93, 'î': 94, 'ï': 95, 'ô': 96, 'ù': 97, 'û': 98, 'œ': 99, '\u2009': 100, '\u200b': 101, '‘': 102, '’': 103, '\u202f': 104}


In [24]:
lines.tar

0                                    \t Va ! \n
1                                 \t Marche. \n
2                                 \t Bouge ! \n
3                                 \t Salut ! \n
4                                  \t Salut. \n
                          ...                  
59995        \t Pourquoi feraient-elles ça ? \n
59996      \t Pourquoi demanderais-tu cela ? \n
59997    \t Pourquoi demanderiez-vous cela ? \n
59998           \t Pourquoi dirais-tu cela ? \n
59999         \t Pourquoi diriez-vous cela ? \n
Name: tar, Length: 60000, dtype: object

In [25]:
lines.tar[0]

'\t Va ! \n'

In [26]:
lines.tar[0][2]

'V'

In [27]:
tar_to_index[lines.tar[0][2]]

48

In [28]:
list(tar_to_index)[47]

'V'

In [29]:
decoder_input = []

for line in lines.tar:
  encoded_line = []
  for char in line:
    encoded_line.append(tar_to_index[char])
  decoder_input.append(encoded_line)

print("샘플 => decoder_input[:5] :", decoder_input[:5])

샘플 => decoder_input[:5] : [[1, 3, 48, 53, 3, 4, 3, 2], [1, 3, 39, 53, 70, 55, 60, 57, 14, 3, 2], [1, 3, 28, 67, 73, 59, 57, 3, 4, 3, 2], [1, 3, 45, 53, 64, 73, 72, 3, 4, 3, 2], [1, 3, 45, 53, 64, 73, 72, 14, 3, 2]]


### 프랑스어 문장의 `\t` 제거
- target data로 사용될 프랑스어 문장의 경우에는 \<sos>에 해당되는 `\t`가 있을 필요가 없다. 따라서 제거해주도록 한다.  

`-` 참고
- `\t`의 인덱스는 `1`이므로 decoder_target을 출력할 경우, 1이 제거된 것을 확인하도록 한다.

In [30]:
decoder_target = []
for line in lines.tar:
  timestep = 0
  encoded_line = []
  for char in line:
    if timestep > 0:
      encoded_line.append(tar_to_index[char])
    timestep = timestep + 1
  decoder_target.append(encoded_line)
print("샘플 => decoder_taget[:5] :", decoder_target[:5])

샘플 => decoder_taget[:5] : [[3, 48, 53, 3, 4, 3, 2], [3, 39, 53, 70, 55, 60, 57, 14, 3, 2], [3, 28, 67, 73, 59, 57, 3, 4, 3, 2], [3, 45, 53, 64, 73, 72, 3, 4, 3, 2], [3, 45, 53, 64, 73, 72, 14, 3, 2]]


## 패딩

`-` 패딩을 위해 가장 길이가 긴 샘플의 길이 확인

In [31]:
max_src_len = max([len(line) for line in lines.src])
max_tar_len = max([len(line) for line in lines.tar])
print("max_src_len :", max_src_len)
print("max_tar_len :", max_tar_len)

max_src_len : 23
max_tar_len : 76


In [32]:
# help(pad_sequences)

In [33]:
encoder_input = pad_sequences(encoder_input, maxlen=max_src_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen=max_tar_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen=max_tar_len, padding='post')

## 원-핫 인코딩

In [34]:
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)

In [35]:
encoder_input

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

# 교사 강요(Teacher forcing)
- 아래의 모든 내용은 `Reference`에서 복사 붙여넣기한 것임을 알려드립니다.  

`-` Reference : https://wikidocs.net/24996  

모델을 설계하기 전에 혹시 의아한 점은 없으신가요? 현재 시점의 디코더 셀의 입력은 오직 이전 디코더 셀의 출력을 입력으로 받는다고 설명하였는데 decoder_input이 왜 필요할까요?

훈련 과정에서는 이전 시점의 디코더 셀의 출력을 현재 시점의 디코더 셀의 입력으로 넣어주지 않고, 이전 시점의 실제값을 현재 시점의 디코더 셀의 입력값으로 하는 방법을 사용할 겁니다. 그 이유는 이전 시점의 디코더 셀의 예측이 틀렸는데 이를 현재 시점의 디코더 셀의 입력으로 사용하면 현재 시점의 디코더 셀의 예측도 잘못될 가능성이 높고 이는 연쇄 작용으로 디코더 전체의 예측을 어렵게 합니다. 이런 상황이 반복되면 훈련 시간이 느려집니다. 만약 이 상황을 원하지 않는다면 이전 시점의 디코더 셀의 예측값 대신 실제값을 현재 시점의 디코더 셀의 입력으로 사용하는 방법을 사용할 수 있습니다. 이와 같이 RNN의 모든 시점에 대해서 이전 시점의 예측값 대신 실제값을 입력으로 주는 방법을 교사 강요라고 합니다.

# seq2seq 기계 번역기 훈련시키기

In [36]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model
import numpy as np

`-` [tensorflow.keras.layers.Input](https://keras.io/api/layers/core_layers/input/)  
`-` [tensorflow.keras.layers.LSTM](https://keras.io/api/layers/recurrent_layers/lstm/)  
`-` [return_sequences_1](https://tykimos.github.io/2017/04/09/RNN_Getting_Started/)  
`-` [return_sequences_2](https://wikidocs.net/106473)  
`-` [Model class](https://keras.io/api/models/model/)  
`-` [model.compile, model.fit, model.predict](https://datascience.stackexchange.com/questions/46124/what-do-compile-fit-and-predict-do-in-keras-sequential-models)

In [95]:
# batch_size를 제외한 shape
encoder_inputs = Input(shape=(None, src_vocab_size))
# units => 은닉 상태 크기, return_state => 인코더의 마지막 내부 상태를 디코더로 넘겨주기 위해 True
encoder_lstm = LSTM(units=256, return_state=True)

# encoder_outputs은 학습할 때 불필요, 나중에 테스트할 때 사용
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)

# LSTM은 바닐라 RNN과는 달리 상태가 두 개. 은닉 상태와 셀 상태.
encoder_states = [state_h, state_c]

In [96]:
decoder_inputs = Input(shape=(None, tar_vocab_size))
# return_sequences =>  Boolean. Whether to return the last output in the output sequence, 
# or the full sequence. Default: False.
decoder_lstm = LSTM(units=256, return_sequences=True, return_state=True)

# 디코더에게 인코더의 은닉 상태, 셀 상태를 전달. => context vector
decoder_outputs, _, _= decoder_lstm(decoder_inputs, initial_state=encoder_states)

decoder_softmax_layer = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

# Model(input, output)
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")

In [97]:
model.fit(x=[encoder_input, decoder_input], y=decoder_target, batch_size=64, epochs=40, validation_split=0.2)

Epoch 1/40
750/750 [==============================] - 37s 45ms/step - loss: 0.7489 - val_loss: 0.6644
Epoch 2/40
750/750 [==============================] - 32s 43ms/step - loss: 0.4596 - val_loss: 0.5396
Epoch 3/40
750/750 [==============================] - 32s 43ms/step - loss: 0.3841 - val_loss: 0.4723
Epoch 4/40
750/750 [==============================] - 32s 43ms/step - loss: 0.3413 - val_loss: 0.4365
Epoch 5/40
750/750 [==============================] - 32s 43ms/step - loss: 0.3123 - val_loss: 0.4108
Epoch 6/40
750/750 [==============================] - 32s 43ms/step - loss: 0.2912 - val_loss: 0.3933
Epoch 7/40
750/750 [==============================] - 33s 43ms/step - loss: 0.2749 - val_loss: 0.3813
Epoch 8/40
750/750 [==============================] - 32s 43ms/step - loss: 0.2617 - val_loss: 0.3706
Epoch 9/40
750/750 [==============================] - 32s 43ms/step - loss: 0.2506 - val_loss: 0.3636
Epoch 10/40
750/750 [==============================] - 32s 43ms/step - loss: 0.241

# seq2seq 기계 번역기 동작시키기

In [98]:
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)

In [99]:
# 이전 시점의 상태들을 저장하는 텐서
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# 문장의 다음 단어를 예측하기 위해서 초기 상태(initial_state)를 이전 시점의 상태로 사용.
# 뒤의 함수 decode_sequence()에 동작을 구현 예정
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)

# 훈련 과정에서와 달리 LSTM의 리턴하는 은닉 상태와 셀 상태를 버리지 않음.
decoder_states = [state_h, state_c]
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)

In [42]:
index_to_src = dict((i, char) for char, i in src_to_index.items())
index_to_tar = dict((i, char) for char, i in tar_to_index.items())

In [110]:
print("src_to_index :", src_to_index)
print("index_to_src :", index_to_src)

src_to_index : {' ': 1, '!': 2, '"': 3, '$': 4, '%': 5, '&': 6, "'": 7, ',': 8, '-': 9, '.': 10, '/': 11, '0': 12, '1': 13, '2': 14, '3': 15, '4': 16, '5': 17, '6': 18, '7': 19, '8': 20, '9': 21, ':': 22, '?': 23, 'A': 24, 'B': 25, 'C': 26, 'D': 27, 'E': 28, 'F': 29, 'G': 30, 'H': 31, 'I': 32, 'J': 33, 'K': 34, 'L': 35, 'M': 36, 'N': 37, 'O': 38, 'P': 39, 'Q': 40, 'R': 41, 'S': 42, 'T': 43, 'U': 44, 'V': 45, 'W': 46, 'X': 47, 'Y': 48, 'Z': 49, 'a': 50, 'b': 51, 'c': 52, 'd': 53, 'e': 54, 'f': 55, 'g': 56, 'h': 57, 'i': 58, 'j': 59, 'k': 60, 'l': 61, 'm': 62, 'n': 63, 'o': 64, 'p': 65, 'q': 66, 'r': 67, 's': 68, 't': 69, 'u': 70, 'v': 71, 'w': 72, 'x': 73, 'y': 74, 'z': 75, '°': 76, 'é': 77, '’': 78, '€': 79}
index_to_src : {1: ' ', 2: '!', 3: '"', 4: '$', 5: '%', 6: '&', 7: "'", 8: ',', 9: '-', 10: '.', 11: '/', 12: '0', 13: '1', 14: '2', 15: '3', 16: '4', 17: '5', 18: '6', 19: '7', 20: '8', 21: '9', 22: ':', 23: '?', 24: 'A', 25: 'B', 26: 'C', 27: 'D', 28: 'E', 29: 'F', 30: 'G', 31: '

In [114]:
def decode_sequence(input_seq):
  # 입력으로부터 인코더의 상태를 얻음
  states_value = encoder_model.predict(input_seq)

  # <SOS>에 해당하는 원-핫 벡터 생성
  target_seq = np.zeros((1, 1, tar_vocab_size))
  target_seq[0, 0, tar_to_index['\t']] = 1.

  stop_condition = False
  decoded_sentence = ""

  # stop_condition이 True가 될 때까지 루프 반복
  while not stop_condition:
    # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
    output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

    # 예측 결과를 문자로 변환
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = index_to_tar[sampled_token_index]

    # 현재 시점의 예측 문자를 예측 문장에 추가
    decoded_sentence += sampled_char

    # <eos>에 도달하거나 최대 길이를 넘으면 중단.
    if (sampled_char == '\n' or
        len(decoded_sentence) > max_tar_len):
        stop_condition = True

    # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
    target_seq = np.zeros((1, 1, tar_vocab_size))
    target_seq[0, 0, sampled_token_index] = 1.

    # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
    states_value = [h, c]

  return decoded_sentence

In [118]:
encoder_input.shape

(60000, 23, 80)

In [115]:
for seq_index in [3,50,100,300,1001]: # 입력 문장의 인덱스
  input_seq = encoder_input[seq_index:seq_index+1]
  decoded_sentence = decode_sequence(input_seq)
  print(35 * "-")
  print('입력 문장:', lines.src[seq_index])
  print('정답 문장:', lines.tar[seq_index][2:len(lines.tar[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
  print('번역 문장:', decoded_sentence[1:len(decoded_sentence)-1]) # '\n'을 빼고 출력

-----------------------------------
입력 문장: Hi.
정답 문장: Salut ! 
번역 문장: Salut. 
-----------------------------------
입력 문장: I see.
정답 문장: Aha. 
번역 문장: Je comprends. 
-----------------------------------
입력 문장: Hug me.
정답 문장: Serrez-moi dans vos bras ! 
번역 문장: Ferme-la. 
-----------------------------------
입력 문장: Help me.
정답 문장: Aidez-moi. 
번역 문장: Aidez-moi ! 
-----------------------------------
입력 문장: I am sure.
정답 문장: Je suis sûr. 
번역 문장: Je suis content. 
